### Here we study the three species telegraphic model of Gene regulation network defined by the following system:

$$
\begin{split}
G &\rightarrow^{k_t} G + P \hspace{20pt} \text{production of the protein P by gene G} \\
G + P &\rightleftharpoons^{k_b}_{k_u} G^* \hspace{20pt} \text{inactivation of the gene G by protein P}\\
P &\rightarrow^{k_{2P}} \emptyset \hspace{20pt} \text{Degradation of protein P}
\end{split}
$$

In [1]:
# push!(LOAD_PATH, "../src/Achedmy/")
using Achedmy
using Pkg
using PyPlot
using Catalyst
using LinearAlgebra
using DifferentialEquations
using Serialization
using LaTeXStrings

In [22]:
        gene_system = @reaction_network begin
            @species G(t) = 0.0 P(t) = 10.0
            @parameters k_on = 0.1 k_off = 1.0 k_p = 10.0 k_d = 1.0
            (k_on, k_off), 0 <--> G
            k_p, G --> G + P
            k_d, P --> 0
        end

Model ##ReactionSystem#274
States (2):
  G(t) [defaults to 0.0]
  P(t) [defaults to 10.0]
Parameters (4):
  k_on [defaults to 0.1]
  k_off [defaults to 1.0]
  k_p [defaults to 10.0]
  k_d [defaults to 1.0]

In [23]:
structure = Achedmy.ReactionStructure(gene_system)
structure.num_reactions

5

In [2]:
bd_system = @reaction_network begin
    @species X(t) = 10.0
    @parameters k_birth = 1.0 k_death = 0.5
    k_birth, 0 --> X
    k_death, X --> 0
end

Model ##ReactionSystem#237
States (1):
  X(t) [defaults to 10.0]
Parameters (2):
  k_birth [defaults to 1.0]
  k_death [defaults to 0.5]

In [7]:
        enzyme_system = @reaction_network begin
            @species S(t) = 1.0 E(t) = 0.9 C(t) = 0.1 X(t) = 0.1
            @parameters k_f = 1.0 k_b = 0.1 k_d = 1.0 k_2X = 1.0 k_2S = 1.0 k_1S = 1.0
            (k_f, k_b), S + E <--> C
            k_d, C --> E + X
            k_2X, X --> 0
            (k_2S, k_1S), S <--> 0
        end

Model ##ReactionSystem#244
States (4):
  S(t) [defaults to 1.0]
  E(t) [defaults to 0.9]
  C(t) [defaults to 0.1]
  X(t) [defaults to 0.1]
Parameters (6):
  k_f [defaults to 1.0]
  k_b [defaults to 0.1]
  k_d [defaults to 1.0]
  k_2X [defaults to 1.0]
⋮

In [ ]:
structure_ez = Achedmy.ReactionStructure(enzyme_system)
structure_ez.num_reactions

11

In [ ]:
structure_ez.stochiometry_prod

4×3 Matrix{Int64}:
 0  1  0
 0  1  1
 1  0  0
 0  0  1

In [13]:
structure_ez.stochiometry_prod[:,2]

4-element Vector{Int64}:
 1
 1
 0
 0

In [11]:
structure_ez.stochiometry_react

4×3 Matrix{Int64}:
 1  0  0
 1  0  0
 0  1  1
 0  0  0

In [3]:
structure_bd = Achedmy.ReactionStructure(bd_system)
structure_bd.num_reactions

2

In [17]:
size(structure_bd.stochiometry_react)

(1, 0)

In [14]:
structure_bd.rate_creation

1-element Vector{Float64}:
 1.0

In [6]:
variables_bd = Achedmy.ReactionVariables(structure_bd,"cross")
sol = Achedmy.solve_dynamics!(structure_bd,variables_bd,selfEnergy="gSBR",atol=atol,
rtol=rtol,tmax=tmax,tstart=tstart);

t: 0.0
t: 1.0e-6
t: 5.999999999999999e-6
t: 3.0999999999999995e-5
t: 8.683191130406408e-5
t: 0.0003112601568662025
t: 0.0012195935514191735
t: 0.0039165652963002815
t: 0.00873142268591634
t: 0.02134450884591095
t: 0.04189591328219638
t: 0.08147459580656584
t: 0.13793539600095084
t: 0.2279711044878183
t: 0.34464437189892444
t: 0.5071238632867029
t: 0.7028626992772737
t: 0.9505194513632611
t: 1.2168111984015297
t: 1.515469156368913
t: 1.8481395547061839
t: 2.207176149270171
t: 2.5939575561452273
t: 3.007080213740854
t: 3.443734655247817
  3.743571 seconds (10.98 M allocations: 563.381 MiB, 4.90% gc time, 99.66% compilation time: <1% of which was recompilation)


In [19]:
gene_system_old = @reaction_network begin
    @species G(t) = 0.01 G_star(t) = 1. P(t) = 0.1 
    @parameters k_t = 10. k_b = 1. k_u = 0.1 k_2P = 2.
   (k_t), G --> G + P
   (k_b,k_u), G + P <--> G_star
   (k_2P), P --> 0 
end

Model ##ReactionSystem#262
States (3):
  G(t) [defaults to 0.01]
  G_star(t) [defaults to 1.0]
  P(t) [defaults to 0.1]
Parameters (4):
  k_t [defaults to 10.0]
  k_b [defaults to 1.0]
  k_u [defaults to 0.1]
  k_2P [defaults to 2.0]

In [10]:
structure_old = Achedmy.ReactionStructure(gene_system_old)
structure_old.num_reactions

9

In [14]:
structure.stochiometry_prod

2×1 Matrix{Int64}:
 1
 1

In [15]:
structure.stochiometry_react

2×1 Matrix{Int64}:
 1
 0

In [4]:
Graph(gene_system)

Graph("G", true, "dot", Catalyst.Statement[Catalyst.Node("G", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("G_star", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("P", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "circle", :color => "#6C9AC3")), Catalyst.Node("rx_1", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "point", :color => "#E28F41", :width => ".1")), Catalyst.Node("rx_2", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "point", :color => "#E28F41", :width => ".1")), Catalyst.Node("rx_3", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "point", :color => "#E28F41", :width => ".1")), Catalyst.Node("rx_4", OrderedCollections.OrderedDict{Symbol, Union{Catalyst.Html, String}}(:shape => "point", :c

In [5]:
#Parameters simulation
tstart= 0.
tmax = 3.5
atol = 1e-3
rtol = 1e-2

0.01

In [6]:
alpha_range = exp10.(range(-2, stop=1, length=10))
println(alpha_range)

[0.01, 0.021544346900318832, 0.046415888336127795, 0.1, 0.2154434690031884, 0.4641588833612779, 1.0, 2.154434690031884, 4.641588833612778, 10.0]


In [ ]:
@unpack k_u = gene_system

SOL = []
VAR = []

for i in alpha_range

    gene_system.defaults[k_u] = i

    structure = Achedmy.ReactionStructure(gene_system)
    variables = Achedmy.ReactionVariables(structure,"cross")
    sol = Achedmy.solve_dynamics!(structure,variables,selfEnergy="gSBR",atol=atol,
    rtol=rtol,tmax=tmax,tstart=tstart);

    push!(SOL,sol)
    push!(VAR,variables)

end

t: 0.0
t: 1.0e-6
t: 5.999999999999999e-6
t: 1.65053789036616e-5
t: 2.971899453036518e-5
t: 7.330958458723329e-5
t: 0.0001745680165972095
t: 0.00038702325473868995
t: 0.0007157062889894946
t: 0.0013268611472890917
t: 0.0021989695066154486
t: 0.0035559271405691114
t: 0.0053576009472853375
t: 0.007855850272509303
t: 0.011019689895548732
t: 0.015069931435849674
t: 0.020037392350625256
t: 0.026054934616192287
t: 0.03327166355705599
t: 0.04168273989906765
t: 0.05160837368767763
t: 0.06286194272707443
t: 0.07598393838611225
t: 0.090563148822556
t: 0.10740866266838756
t: 0.125839202939621
t: 0.14698246155018013
t: 0.16983617394860845
t: 0.19589991939311774
t: 0.22379228628035808
t: 0.25366049298567417
t: 0.2859515270655994
t: 0.3206224789838324
t: 0.3578110283271684
t: 0.3976721459861596
t: 0.4403172081570116
t: 0.48587535295901585
t: 0.5344748574693021
t: 0.5862375950882389
t: 0.6412833930547774
t: 0.6997269238305224
t: 0.7616760273061457
t: 0.8272308281102231
t: 0.896482207345653
t: 0.969510

In [ ]:
# # Saving data

# open("../data/gene/sol_gSBR.jls", "w") do f
#     serialize(f, SOL)
# end
# open("../data/gene/var_gSBR.jls", "w") do f
#     serialize(f, VAR)
# end

In [ ]:
# Reading data

open("../data/gene/sol_gSBR.jls", "r") do f
    global SOL = deserialize(f)
end

open("../data/gene/var_gSBR.jls", "r") do f
    global VAR = deserialize(f)
end

In [ ]:
@unpack k_u = gene_system

SOL_crossMCA = []
VAR_crossMCA = []

for i in alpha_range

    gene_system.defaults[k_u] = i

    structure = Achedmy.ReactionStructure(gene_system)
    variables = Achedmy.ReactionVariables(structure,"cross")
    sol = Achedmy.solve_dynamics!(structure,variables,selfEnergy="MCA",atol=atol,
    rtol=rtol,tmax=tmax,tstart=tstart);

    push!(SOL_crossMCA,sol)
    push!(VAR_crossMCA,variables)

end

In [ ]:
# # Saving data

# open("../data/gene/sol_crossMCA.jls", "w") do f
#     serialize(f, SOL_crossMCA)
# end
# open("../data/gene/var_crossMCA.jls", "w") do f
#     serialize(f, VAR_crossMCA)
# end

In [ ]:
# Reading data
open("../data/gene/sol_crossMCA.jls", "r") do f
    global SOL_crossMCA = deserialize(f)
end
open("../data/gene/var_crossMCA.jls", "r") do f
    global VAR_crossMCA = deserialize(f)
end

In [ ]:
@unpack k_u = gene_system

SOL_singleMCA = []
VAR_singleMCA = []

for i in alpha_range

    try
        gene_system.defaults[k_u] = i

        structure = Achedmy.ReactionStructure(gene_system)
        variables = Achedmy.ReactionVariables(structure,"single")
        sol = Achedmy.solve_dynamics!(structure,variables,selfEnergy="MCA",atol=atol,
        rtol=rtol,tmax=tmax,tstart=tstart);

        push!(SOL_singleMCA,sol)
        push!(VAR_singleMCA,variables)
    catch e
        println("Error for k_u = $i: $e")
    end

end

In [ ]:
# # Saving data
# open("../data/gene/sol_singleMCA.jls", "w") do f
#     serialize(f, SOL_singleMCA)
# end
# open("../data/gene/var_singleMCA.jls", "w") do f
#     serialize(f, VAR_singleMCA)
# end

In [ ]:
# # Reading data
open("../data/gene/sol_singleMCA.jls", "r") do f
    global SOL_singleMCA = deserialize(f)
end
open("../data/gene/var_singleMCA.jls", "r") do f
    global VAR_singleMCA = deserialize(f)
end

In [ ]:
@unpack k_u = gene_system

SOL_singleSBR = []
VAR_singleSBR = []

for i in alpha_range

    try
        gene_system.defaults[k_u] = i

        structure = Achedmy.ReactionStructure(gene_system)
        variables = Achedmy.ReactionVariables(structure,"single")
        sol = Achedmy.solve_dynamics!(structure,variables,selfEnergy="SBR",atol=atol,
        rtol=rtol,tmax=tmax,tstart=tstart);

        push!(SOL_singleSBR,sol)
        push!(VAR_singleSBR,variables)
    catch e
        println("Error for k_u = $i: $e")
    end

end

In [ ]:
# # Saving data
# open("../data/gene/sol_singleSBR.jls", "w") do f
#     serialize(f, SOL_singleSBR)
# end
# open("../data/gene/var_singleSBR.jls", "w") do f
#     serialize(f, VAR_singleSBR)
# end

In [ ]:
# # Reading data
open("../data/gene/sol_singleSBR.jls", "r") do f
    global SOL_singleSBR = deserialize(f)
end
open("../data/gene/var_singleSBR.jls", "r") do f
    global VAR_singleSBR = deserialize(f)
end

In [ ]:
@unpack k_u = gene_system

SOL_single_gSBR = []
VAR_single_gSBR = []

for i in alpha_range

    try
        gene_system.defaults[k_u] = i

        structure = Achedmy.ReactionStructure(gene_system)
        variables = Achedmy.ReactionVariables(structure,"single")
        sol = Achedmy.solve_dynamics!(structure,variables,selfEnergy="gSBR",atol=atol,
        rtol=rtol,tmax=tmax,tstart=tstart);

        push!(SOL_single_gSBR,sol)
        push!(VAR_single_gSBR,variables)
    catch e
        println("Error for k_u = $i: $e")
    end

end

In [ ]:
# # Saving data
# open("../data/gene/sol_single_gSBR.jls", "w") do f
#     serialize(f, SOL_single_gSBR)
# end
# open("../data/gene/var_single_gSBR.jls", "w") do f
#     serialize(f, VAR_single_gSBR)
# end

In [ ]:
# # Reading data
open("../data/gene/sol_single_gSBR.jls", "r") do f
    global SOL_single_gSBR = deserialize(f)
end
open("../data/gene/var_single_gSBR.jls", "r") do f
    global VAR_single_gSBR = deserialize(f)
end

### Plot the means:

In [ ]:
plot_indices = [1,4,7,10]
figure(figsize=(10,9))
for i in range(1,length(plot_indices))
    subplot(2,2,i)
    for j in range(1,length(species(gene_system)))
        plot(SOL[plot_indices[i]].t,VAR[plot_indices[i]].μ[j,:],label=alpha_range[plot_indices[i]])
        legend(["G","G*","P"])
    end
    title(L"$k_u$ = " * string(alpha_range[plot_indices[i]]))
    yscale("log")
    xlabel(L"$\tau$")
    ylabel(L"$\langle n(\tau)\rangle$")
end
gcf()

In [ ]:
close("all")

### Plot the variances:

In [ ]:
#plot_indices = [1,4,7,10]
figure(figsize=(10,9))
for i in range(1,length(plot_indices))
    subplot(2,2,i)
    for j in range(1,length(species(gene_system)))
        plot(SOL[plot_indices[i]].t,VAR[plot_indices[i]].μ[j,:] + diag(VAR[plot_indices[i]].C[j,j,:,:]),label=alpha_range[plot_indices[i]])
        legend(["G","G*","P"])
    end
    title(L"$k_u$ = " * string(alpha_range[plot_indices[i]]))
    yscale("log")
    xlabel(L"$\tau$")
    ylabel(L"$\langle \delta n(\tau) \delta n(\tau) \rangle$")
end

gcf()

In [ ]:
close()

### Plot the time-step sizes:


In [ ]:
plot_indices = [1,4,7,10]
figure(figsize=(5,4))
end_idx = [length(SOL[i].t) - 1 for i in 1:length(alpha_range)]
for i in range(1,length(plot_indices))
    scatter(SOL[plot_indices[i]].t,SOL[plot_indices[i]].w[end],label=alpha_range[plot_indices[i]],s=5)
end
legend(title = L"$k_u$")
title("gSBR time-step size for gene regulation system")
yscale("log")
xscale("log")
xlabel(L"$\tau$")
ylabel(L"$\Delta \tau$")
gcf()

In [ ]:
plt.savefig("../plots/gene/time_step_loglog.png",bbox_inches="tight")
plt.close()

In [ ]:
plot_indices = [1,4,7,10]
figure(figsize=(5,4))
end_idx = [length(SOL[i].t) - 1 for i in 1:length(alpha_range)]
for i in range(1,length(plot_indices))
    scatter(SOL[plot_indices[i]].t,SOL[plot_indices[i]].w[end],label=alpha_range[plot_indices[i]],s=5)
end
legend(title = L"$k_u$")
title("gSBR time-step size for gene regulation system")
# yscale("log")
# xscale("log")
xlabel(L"$\tau$")
ylabel(L"$\Delta \tau$")
gcf()

In [ ]:
plt.savefig("../plots/gene/time_step.png",bbox_inches="tight")
plt.close()

In [ ]:
## Running MAK:
@unpack k_u = gene_system

SOL_mak = []
VAR_mak = []

for i in alpha_range
    gene_system.defaults[k_u] = i
    println(gene_system.defaults)

    structure = Achedmy.ReactionStructure(gene_system)
    variables = Achedmy.ReactionVariables(structure,"single")
    sol = Achedmy.solve_dynamics!(structure,variables,selfEnergy="MAK",atol=atol,
    rtol=rtol,tmax=tmax,tstart=tstart);

    push!(SOL_mak,sol)
    push!(VAR_mak,variables)

end

In [ ]:
# ## Saving data

# open("../data/gene/sol_mak.jls", "w") do f
#     serialize(f, SOL_mak)
# end

# open("../data/gene/var_mak.jls", "w") do f
#     serialize(f, VAR_mak)
# end


In [ ]:
# ## Reading data
open("../data/gene/sol_mak.jls", "r") do f
    global SOL_mak = deserialize(f)
end
open("../data/gene/var_mak.jls", "r") do f
    global VAR_mak = deserialize(f)
end

### Plotting the cross variances $\langle \delta n_i(\tau)\delta n_j(\tau) \rangle$:

In [ ]:
fig1 = figure(figsize=(15,8))

ax_3 = fig1.add_axes([0.,0, 0.3, 0.3])
ax_4 = fig1.add_axes([0.35,0, 0.3, 0.3])

ax_1 = fig1.add_axes([0.,0.35, 0.3, 0.3])
ax_2 = fig1.add_axes([0.35,0.35, 0.3, 0.3])

ax_5 = fig1.add_axes([0.7,0.35, 0.3, 0.3])

ax_6 = fig1.add_axes([0.7,0., 0.3, 0.3])

ax_11 = fig1.add_axes([0.,0.7, 0.3, 0.3])
ax_22 = fig1.add_axes([0.35,0.7, 0.3, 0.3])

ax_55 = fig1.add_axes([0.7,0.7, 0.3, 0.3])

axes = [ax_1, ax_2, ax_5, ax_3, ax_4, ax_6, ax_11, ax_22, ax_55]
sat = [0, 3, 6] .+ 1
skip = 1
cOL = plt.rcParams["axes.prop_cycle"].by_key()["color"]

for i in 1:length(sat)
    
    axes[i].plot(SOL[sat[i]].t[1:skip:end], diag(VAR[sat[i]].C[1,3,1:skip:end,1:skip:end]), linestyle="-", c=cOL[1], alpha=1., linewidth=3)

    axes[i].spines["right"].set_visible(false)
    axes[i].spines["top"].set_visible(false)
    axes[i].set_ylim((-0.055, 0.75))
    axes[i].set(xticklabels=[])
    if i != 1
        axes[i].set(yticklabels=[])
    end
end

axes[1].set_ylabel(L"\langle \delta n_G(\tau)\delta n_P(\tau) \rangle", fontsize=24)

for i in 1:length(sat)
    axes[i+3].plot(SOL[sat[i]].t[1:skip:end], diag(VAR[sat[i]].C[2,3,1:skip:end,1:skip:end]), linestyle="-", c=cOL[1], alpha=1., linewidth=3)

    axes[i+3].spines["right"].set_visible(false)
    axes[i+3].spines["top"].set_visible(false)
    axes[i+3].set_xlabel(L"\tau", fontsize=22)
    axes[i+3].set_ylim((-0.055, 0.65))
    if i != 1
        axes[i+3].set(yticklabels=[])
    end
end

axes[4].set_ylabel(L"\langle \delta n_{G^*}(\tau)\delta n_P(\tau) \rangle", fontsize=24)

for i in 1:length(sat)
    axes[i+6].plot(SOL[sat[i]].t[1:skip:end], diag(VAR[sat[i]].C[1,2,1:skip:end,1:skip:end]), linestyle="-", c=cOL[1], alpha=1., linewidth=3)

    axes[i+6].spines["right"].set_visible(false)
    axes[i+6].spines["top"].set_visible(false)
    axes[i+6].set(xticklabels=[])
    axes[i+6].set_title("\$k_u\$ = " * string(round(alpha_range[sat[i]], digits=2)), fontsize=20)
    axes[i+6].set_ylim((-0.027, 0.04))
    if i != 1
        axes[i+6].set(yticklabels=[])
    end
end

axes[7].set_ylabel(L"\langle \delta n_G(\tau)\delta n_{G^*}(\tau) \rangle", fontsize=24)

ax_1.legend(["gSBR"], bbox_to_anchor=(1., 0.75), loc=1, frameon=false, fontsize=16, ncol=1)

gcf()


In [ ]:
close()

### Plotting the two-time correlator PP $\langle \delta n_P(\tau) \delta n_P(\tau^\prime) \rangle$ :

In [ ]:
function find_closed_timegrid_indices(timegrid, times)
    return [findmin(abs.(timegrid .- X))[2] for X in times]
end

In [ ]:
sat = 4

time_grid_gSBR = SOL[sat].t
plotting_tau =  [0.75, 1.5, 2.25, 3.0]

tau_SBR = find_closed_timegrid_indices(time_grid_gSBR, plotting_tau)

fig = figure(figsize=(15, 3.5))
ax1 = fig.add_axes([0, 0., 0.1, 1])
ax2 = fig.add_axes([0.11, 0., 0.19, 1])
ax3 = fig.add_axes([0.31, 0., 0.29, 1])
ax4 = fig.add_axes([0.61, 0., 0.38, 1])

axis = [ax4, ax3, ax2, ax1]

for j in 1:length(tau_SBR)
    
    axis[j].plot(time_grid_gSBR[tau_SBR[j]:end] .- time_grid_gSBR[tau_SBR[j]], VAR[sat].N[3, 3, tau_SBR[j]:end, tau_SBR[j]], "-", alpha=1.,linewidth=3)

    axis[j].set_title(L"\tau^\prime = " * string(plotting_tau[j]))
    axis[j].spines["right"].set_visible(false)
    axis[j].spines["top"].set_visible(false)
    axis[j].set_xlabel(L"\tau-\tau\prime", fontsize=22)
    axis[j].set_ylim((-0.1, 1))
end

ax1.set_ylabel(L"\langle \delta n_P(\tau) \delta n_P(\tau^\prime) \rangle ", fontsize=24)
ax2.set(yticklabels=[])
ax3.set(yticklabels=[])
ax4.set(yticklabels=[])

ax4.legend(["gSBR"], bbox_to_anchor=(0.5, 0.45), loc=0, frameon=false, fontsize=14)

gcf()

In [ ]:
close("all")

### Plotting the response: $R_{PP}(\tau,\tau')$

In [ ]:
sat = 6

time_grid_gSBR = SOL[sat].t
plotting_tau =  [0.75, 1.5, 2.25, 3.0]

tau_SBR = find_closed_timegrid_indices(time_grid_gSBR, plotting_tau)

fig = figure(figsize=(15, 3.5))
ax1 = fig.add_axes([0, 0., 0.1, 1])
ax2 = fig.add_axes([0.11, 0., 0.19, 1])
ax3 = fig.add_axes([0.31, 0., 0.29, 1])
ax4 = fig.add_axes([0.61, 0., 0.38, 1])

axis = [ax4, ax3, ax2, ax1]

for j in 1:length(plotting_tau)

    axis[j].plot(time_grid_gSBR[tau_SBR[j]:end] .- time_grid_gSBR[tau_SBR[j]],(VAR[sat].R[3, 3, tau_SBR[j]:end, tau_SBR[j]]), "-", alpha=0.75,linewidth=3)
    
    axis[j].set_title(L"\tau^\prime = " * string(plotting_tau[j]), fontsize=22)
    axis[j].spines["right"].set_visible(false)
    axis[j].spines["top"].set_visible(false)
    axis[j].set_xlabel(L"\tau-\tau\prime", fontsize=22)
    axis[j].set_ylim((-0.1, 1.))
end

ax1.set_ylabel(L"R_{PP}(\tau,\tau^\prime)", fontsize=22)
ax2.set(yticklabels=[])
ax3.set(yticklabels=[])
ax4.set(yticklabels=[])

ax4.legend(["gSBR"], bbox_to_anchor=(0.45, 0.95), loc=0, frameon=false, fontsize=18)

gcf()


In [ ]:
close("all")

## Compare with other dynamical methods:

Note that now we use other libraries and code written in python for the implementation of other dynamical methods. They are NOT originally intended to be a part of the package.

In [ ]:
using PyCall
using Pkg
using Conda

py"""
import sys
sys.path.append("../extras/other_dynamics")
"""

@pyimport cheMASTER
@pyimport emre

@pyimport numpy as np
@pyimport importlib

In [ ]:
# Setting up the reactions and the parameters:

num_species     = 3 #Number of species in the reaction system
num_interaction = 3 # Number of interating reactions in the Hamiltonian
num_reactions   = 2*num_species + num_interaction # This counts the total number of possible reactions in the system.
                                        # We have 2N creation and destruction reactions and the rest interaction reactions

# We take the order to be G,G*,P

# Rate constants for the reaction system: [k1 is Creation and k2 is Destruction]
k1 = np.array([0.,0.,0.]) #len must be equal to num_species
k2 = np.array([0.,0.,2.]) #len must be equal to num_species

k3 = np.array([10,1.,0.1]) 
s_i,r_i = np.zeros([num_interaction,num_species]),np.zeros([num_interaction,num_species])
rxn_par = [k1,k2,k3]

s_i[1,:] = [1,0,1] 
r_i[1,:] = [1,0,0]
s_i[2,:] = [0,1,0]
r_i[2,:] = [1,0,1]
s_i[3,:] = [1,0,1]
r_i[3,:] = [0,1,0]

#The initial concentration of the species. 
#Note: for Gillespie we start each trajectory from a Poisonian random variable with mean as the initial conc.below:
initial_values = np.array([0.01,1.,0.1])

dt = 0.05
tf = 3.5

In [ ]:
#Run some Gillespie simulations
num_gill = 100
gill     = []

for i in range(1,num_gill)
    push!(gill,cheMASTER.chemical_system_class(num_interaction,num_species,rxn_par,r_i,s_i))
    cheMASTER.gillespie_avg(gill[i],1,initial_values,0,tf,dt,10000)
    print(gill[i])
end

In [ ]:
#Plot settings!

cmap = plt.get_cmap("tab10")
col = [cmap(i) for i in np.linspace(0, 1, num_gill-75)]

original_rcParams = copy(plt.rcParams)
rcParams = PyDict(matplotlib["rcParams"])

plot_params = Dict("font.size" => 70, "font.family" => "sans-serif", "text.usetex" => true, "font.weight" => 900, "axes.linewidth" => 2, "xtick.major.width" => 3, "xtick.major.size" => 5, "xtick.minor.width" => 2, "xtick.minor.size" => 3, "ytick.major.width" => 3, "ytick.major.size" => 5, "ytick.minor.width" => 2, "ytick.minor.size" => 3, "axes.labelsize" => 18, "xtick.labelsize" => 18, "ytick.labelsize" => 18)
plot_params2 = Dict("text.usetex" => false, "xtick.labelsize" => 20, "ytick.labelsize" => 20, "xtick.major.width" => 3, "ytick.major.width" => 3, "xtick.major.size" => 10, "ytick.major.size" => 10, "xtick.direction" => "in", "ytick.direction" => "in", "font.size" => 30, "figure.autolayout" => true, "figure.figsize" => (10, 5), "axes.titlesize" => 20, "axes.labelsize" => 20, "lines.linewidth" => 3, "lines.markersize" => 3, "legend.fontsize" => 13, "mathtext.fontset" => "stix", "axes.linewidth" => 3)

merge!(rcParams,plot_params)
merge!(rcParams,plot_params2)

In [ ]:

fig1  = plt.figure(figsize=(15,8))

ax_1 = fig1.add_axes([0.,0.525, 0.3, 0.4])
ax_2 = fig1.add_axes([0.35,0.525, 0.3, 0.4])
ax_3 = fig1.add_axes([0.7,0.525, 0.3, 0.4])

axes = [ax_1,ax_2,ax_3]
sat  = [0,3,6]
skip = 2
cOL = plt.rcParams["axes.prop_cycle"].by_key()["color"]

for i in range(1,num_species)
    for j in range(1,num_gill-75)
        if all(gill[j].y[1,:] .- 3 .< 0.)
            #axes[i].plot(gill[j].timeGrid, gill[j].y[i,:], "--", c=colors[j], alpha=1., linewidth=3)
            axes[i].plot(gill[j].timeGrid[1:skip:end], gill[j].y[i,1:skip:end], "--", c=col[j], alpha=1., linewidth=3)
        end
    end
    axes[i].spines["right"].set_visible(false)
    axes[i].spines["top"].set_visible(false)
    #axes[i].set_title("Enzyme Sat = "+str(np.round(enzyme_sat[sat[i]],2)))
    #axes[i].set_title("$k_u$ = "+str(np.round(alpha_range[sat[i]],2)),fontsize=20)
    axes[i].set_xlabel(L"${\tau}$",fontsize=22)
end
axes[1].set_ylabel(L"$n_G$",fontsize=28)
axes[2].set_ylabel(L"$n_{G^*}$",fontsize=28)
axes[3].set_ylabel(L"$n_P$",fontsize=28)

gcf()

In [ ]:
plt.savefig("../plots/gene/gene_gillespie_new.png",bbox_inches="tight")
plt.close()

## Run Master equation and EMRE solutions:

In [ ]:
Master = []

for i in range(1,length(alpha_range))
    
    rxn_par[3][3] = alpha_range[i]

    push!(Master,cheMASTER.chemical_system_class(num_interaction,num_species,rxn_par,r_i,s_i))
    cheMASTER.initialize_dynamics(Master[i],initial_values,0,tf,dt)
    cheMASTER.runDynamics(Master[i],max_num=[10,10,20],method = "Eigenvalue",crossCorrelator_tau="connected",variance=true)

    println(i)
end

In [ ]:
# open("../data/gene/master.jls", "w") do f
#     serialize(f, Master)
# end

In [ ]:
open("../data/gene/master.jls", "r") do f
    global Master = deserialize(f)
end

In [ ]:
EMRE   = []

for i in range(1,length(alpha_range))
    
    rxn_par[3][3] = alpha_range[i]
    push!(EMRE,emre.chemical_system_class(num_interaction,num_species,rxn_par,r_i,s_i))
    emre.initialize_dynamics(EMRE[i],initial_values,0,tf,dt,alpha=1.,volume=1.,measureResponse=true,crossCorrelator_tau=true)
    emre.runDynamics(EMRE[i])

    println(i)
end

In [ ]:
# open("../data/gene/emre.jls", "w") do f
#     serialize(f, EMRE)
# end

In [ ]:
open("../data/gene/emre.jls", "r") do f
    global EMRE = deserialize(f)
end

In [ ]:
plot_params3 = (Dict(
    "font.size" => 20,
    "font.family" => "sans-serif",
    "text.usetex" => true,
    "font.weight" => 900,
    "axes.linewidth" => 2,
    "xtick.major.width" => 3,
    "xtick.major.size" => 5,
    "xtick.minor.width" => 2,
    "xtick.minor.size" => 3,
    "ytick.major.width" => 3,
    "ytick.major.size" => 5,
    "ytick.minor.width" => 2,
    "ytick.minor.size" => 3,
    "axes.labelsize" => 18,
    "xtick.labelsize" => 18,
    "ytick.labelsize" => 18,
    "font.size" => 20,
))

#rcParams = PyDict(matplotlib["rcParams"])
merge!(rcParams, plot_params3)

fig1 = plt.figure(figsize=(15, 8))

ax_3 = fig1.add_axes([0., 0, 0.3, 0.4])
ax_4 = fig1.add_axes([0.35, 0, 0.3, 0.4])

ax_1 = fig1.add_axes([0., 0.525, 0.3, 0.4])
ax_2 = fig1.add_axes([0.35, 0.525, 0.3, 0.4])

ax_5 = fig1.add_axes([0.7, 0.525, 0.3, 0.4])

ax_6 = fig1.add_axes([0.7, 0., 0.3, 0.4])

axes = [ax_1, ax_2, ax_5, ax_3, ax_4, ax_6]
sat = [0, 3, 6] .+ 1
skip = 1
cOL = plt.rcParams["axes.prop_cycle"].by_key()["color"]

for i in 1:length(sat)
    for j in 1:num_species
        #axes[i].plot(QUAD2SBR[sat[i]].timeGrid[1:skip:end], QUAD2SBR[sat[i]].y[j, 1:skip:end]', linestyle="-", c=cOL[j], alpha=1., linewidth=3)
        axes[i].plot(SOL[sat[i]].t, VAR[sat[i]].μ[j, :], linestyle="-", c=cOL[j], alpha=1., linewidth=3)

        #axes[i].plot(MAK[sat[i]].timeGrid[1:skip:end], MAK[sat[i]].y[j, 1:skip:end]', linestyle=(0, (1, 1)), c=cOL[j], alpha=1., linewidth=3)
        axes[i].plot(SOL_mak[sat[i]].t, VAR_mak[sat[i]].μ[j, :], linestyle=(0, (1, 1)), c=cOL[j], alpha=1., linewidth=3)
        
        axes[i].plot(Master[sat[i]].timeGrid[1:skip:end], Master[sat[i]].y[j, 1:skip:end]', "--", c=cOL[j], alpha=1., linewidth=3)
    end
    axes[i].spines["right"].set_visible(false)
    axes[i].spines["top"].set_visible(false)
    axes[i].set_title("\$k_u\$ = " * string(round(alpha_range[sat[i]], digits=2)), fontsize=20)
    axes[i].set_yscale("log")
    axes[i].set_ylim((0.002, 2.50))
    axes[i].set(xticklabels=[])
    if i != 1
        axes[i].set(yticklabels=[])
    end
end
axes[1].set_ylabel(L"$ \langle n(\tau) \rangle $", fontsize=24)

for i in 1:length(sat)
    for j in 1:num_species
        #axes[i + 3].plot(QUAD2SBR[sat[i]].timeGrid[1:skip:end], (QUAD2SBR[sat[i]].y[j, 1:skip:end] + diag(QUAD2SBR[sat[i]].corr[j, j, 1:skip:end, 1:skip:end]))', linestyle="-", c=cOL[j], alpha=1., linewidth=3)
        
        axes[i + 3].plot(SOL[sat[i]].t, (VAR[sat[i]].μ[j,:] .+ diag(VAR[sat[i]].C[j, j, :, :])), linestyle="-", c=cOL[j], alpha=1., linewidth=3)
        #[TODO] Change this! The variance doesn't exactly match right now!
        #axes[i + 3].plot(SOL[sat[i]].t, (VAR[sat[i]].μ[j,:]), linestyle="-", c=cOL[j], alpha=1., linewidth=3)
        axes[i + 3].plot(EMRE[sat[i]].timeGrid[1:skip:end], EMRE[sat[i]].lna_var[j, j, 1:skip:end]', linestyle="dotted", c=cOL[j], alpha=1., linewidth=3)
        axes[i + 3].plot(Master[sat[i]].timeGrid[1:skip:end], Master[sat[i]].variance[j, 1:skip:end]', "--", c=cOL[j], alpha=1., linewidth=3)
    end
    axes[i + 3].spines["right"].set_visible(false)
    axes[i + 3].spines["top"].set_visible(false)
    axes[i + 3].set_xlabel(L"$ \tau $", fontsize=22)
    axes[i + 3].set_yscale("log")
    axes[i + 3].set_ylim((0.002, 5.))
    if i != 1
        axes[i + 3].set(yticklabels=[])
    end
end
axes[4].set_ylabel(L"$\langle \delta n(\tau)\delta n(\tau) \rangle $", fontsize=24)

axes[1].legend(["G, gSBR", "G, MAK", "G, Master", "G*, gSBR", "G*, MAK", "G*, Master", "P, gSBR", "P, MAK", "P, Master"], bbox_to_anchor=(1.06, 0.89), loc=1, frameon=false, fontsize=13, ncol=3)
axes[6].legend(["G, gSBR","G, LNA","G, Master","G*, gSBR","G*, LNA","G*, Master","P, gSBR","P, LNA","P, Master"],bbox_to_anchor=(1.1, 0.35), loc=1, frameon=false, fontsize=13,ncol=3)

gcf()

In [ ]:
plt.savefig("../plots/gene/gene_means_variance.png", bbox_inches="tight")
plt.close()


In [ ]:
sat = 4

time_grid_all = EMRE[sat].timeGrid
time_grid_gSBR = SOL[sat].t
plotting_tau =  [0.75, 1.5, 2.25, 3.0]

tau = find_closed_timegrid_indices(time_grid_all, plotting_tau)
tau_SBR = find_closed_timegrid_indices(time_grid_gSBR, plotting_tau)

fig = figure(figsize=(15, 3.5))
ax1 = fig.add_axes([0, 0., 0.1, 1])
ax2 = fig.add_axes([0.11, 0., 0.19, 1])
ax3 = fig.add_axes([0.31, 0., 0.29, 1])
ax4 = fig.add_axes([0.61, 0., 0.38, 1])

axis = [ax4, ax3, ax2, ax1]

for j in 1:length(tau)
        
    axis[j].plot(time_grid_gSBR[tau_SBR[j]:end] .- time_grid_gSBR[tau_SBR[j]], VAR[sat].N[3, 3, tau_SBR[j]:end, tau_SBR[j]], "-", alpha=0.75, linewidth=3)

    axis[j].plot(time_grid_all[1:end-tau[j]], EMRE[sat].corr[3, 3, tau[j]+1:end, tau[j]], linestyle="dashdot", alpha=0.75,linewidth=3)

    axis[j].plot(Master[sat].timeGrid[1:end-tau[j]], Master[sat].correlator_tau[3, 3, 1:end-tau[j], tau[j]], "--", alpha=0.75, color="gray",linewidth=3)

    axis[j].set_title(L"\tau^\prime = " * string(plotting_tau[j]))
    axis[j].spines["right"].set_visible(false)
    axis[j].spines["top"].set_visible(false)
    axis[j].set_xlabel(L"\tau-\tau\prime", fontsize=22)
    axis[j].set_ylim((-0.1, 2))
end

ax1.set_ylabel(L"\langle \delta n_P(\tau) \delta n_P(\tau^\prime) \rangle ", fontsize=24)
ax2.set(yticklabels=[])
ax3.set(yticklabels=[])
ax4.set(yticklabels=[])

ax4.legend(["gSBR", "LNA", "Master"], bbox_to_anchor=(0.5, 0.45), loc=0, frameon=false, fontsize=14)

gcf()

In [ ]:
plt.savefig("../plots/gene/gene_num_corr_P.png", bbox_inches="tight")
plt.close()

In [ ]:
fig1 = figure(figsize=(15,8))

ax_3 = fig1.add_axes([0.,0, 0.3, 0.3])
ax_4 = fig1.add_axes([0.35,0, 0.3, 0.3])

ax_1 = fig1.add_axes([0.,0.35, 0.3, 0.3])
ax_2 = fig1.add_axes([0.35,0.35, 0.3, 0.3])

ax_5 = fig1.add_axes([0.7,0.35, 0.3, 0.3])

ax_6 = fig1.add_axes([0.7,0., 0.3, 0.3])

ax_11 = fig1.add_axes([0.,0.7, 0.3, 0.3])
ax_22 = fig1.add_axes([0.35,0.7, 0.3, 0.3])

ax_55 = fig1.add_axes([0.7,0.7, 0.3, 0.3])

axes = [ax_1, ax_2, ax_5, ax_3, ax_4, ax_6, ax_11, ax_22, ax_55]
sat = [0, 3, 6] .+ 1
skip = 1
cOL = plt.rcParams["axes.prop_cycle"].by_key()["color"]

for i in 1:length(sat)
    
    axes[i].plot(SOL[sat[i]].t[1:skip:end], diag(VAR[sat[i]].N[1,3,1:skip:end,1:skip:end]), linestyle="-", c=cOL[1], alpha=1., linewidth=3)
    axes[i].plot(EMRE[sat[i]].timeGrid[1:skip:end], EMRE[sat[i]].lna_var[1,3,1:skip:end], linestyle="dashdot", c=cOL[2], alpha=1., linewidth=3)
    axes[i].plot(Master[sat[i]].timeGrid[1:end], Master[sat[i]].correlator_tau[1,3,1,1:end], "--", c="gray", alpha=1., linewidth=3)
    #axes[i].plot(Master[sat[i]].timeGrid[1:skip:end], Master[sat[i]].correlator_tau[1,3,1:skip:end,1], "--", c="gray", alpha=1., linewidth=3)


    axes[i].spines["right"].set_visible(false)
    axes[i].spines["top"].set_visible(false)
    axes[i].set_ylim((-0.055, 0.75))
    axes[i].set(xticklabels=[])
    if i != 1
        axes[i].set(yticklabels=[])
    end
end

axes[1].set_ylabel(L"\langle \delta n_G(\tau)\delta n_P(\tau) \rangle", fontsize=24)

for i in 1:length(sat)
    axes[i+3].plot(SOL[sat[i]].t[1:skip:end], diag(VAR[sat[i]].N[2,3,1:skip:end,1:skip:end]), linestyle="-", c=cOL[1], alpha=1., linewidth=3)
    axes[i+3].plot(EMRE[sat[i]].timeGrid[1:skip:end], EMRE[sat[i]].lna_var[2,3,1:skip:end], linestyle="dashdot", c=cOL[2], alpha=1., linewidth=3)
    axes[i+3].plot(Master[sat[i]].timeGrid[1:end], Master[sat[i]].correlator_tau[2,3,1,1:end], "--", c="gray", alpha=1., linewidth=3)
    #axes[i+3].plot(Master[sat[i]].timeGrid[1:skip:end], Master[sat[i]].correlator_tau[2,3,1:skip:end,1], "--", c="gray", alpha=1., linewidth=3)


    axes[i+3].spines["right"].set_visible(false)
    axes[i+3].spines["top"].set_visible(false)
    axes[i+3].set_xlabel(L"\tau", fontsize=22)
    axes[i+3].set_ylim((-0.055, 0.65))
    if i != 1
        axes[i+3].set(yticklabels=[])
    end
end

axes[4].set_ylabel(L"\langle \delta n_{G^*}(\tau)\delta n_P(\tau) \rangle", fontsize=24)

for i in 1:length(sat)
    axes[i+6].plot(SOL[sat[i]].t[1:skip:end], diag(VAR[sat[i]].N[1,2,1:skip:end,1:skip:end]), linestyle="-", c=cOL[1], alpha=1., linewidth=3)
    axes[i+6].plot(EMRE[sat[i]].timeGrid[1:skip:end], EMRE[sat[i]].lna_var[1,2,1:skip:end], linestyle="dashdot", c=cOL[2], alpha=1., linewidth=3)
    axes[i+6].plot(Master[sat[i]].timeGrid[1:skip:end], Master[sat[i]].correlator_tau[1,2,1,1:end], "--", c="gray", alpha=1., linewidth=3)

    axes[i+6].spines["right"].set_visible(false)
    axes[i+6].spines["top"].set_visible(false)
    axes[i+6].set(xticklabels=[])
    axes[i+6].set_title("\$k_u\$ = " * string(round(alpha_range[sat[i]], digits=2)), fontsize=20)
    axes[i+6].set_ylim((-0.027, 0.04))
    if i != 1
        axes[i+6].set(yticklabels=[])
    end
end

axes[7].set_ylabel(L"\langle \delta n_G(\tau)\delta n_{G^*}(\tau) \rangle", fontsize=24)

ax_1.legend(["gSBR", "LNA", "Master"], bbox_to_anchor=(1., 0.75), loc=1, frameon=false, fontsize=16, ncol=1)

gcf()


In [ ]:
plt.savefig("../plots/gene/gene_equal_time_corr.png", bbox_inches="tight")
plt.close()

In [ ]:
Master_response = []

#This is calculation intensive, be careful!

for i in range(1,length(alpha_range))
    
    rxn_par[3][3] = alpha_range[i]
    rxn_par[1]    = rxn_par[1] .+ 0.001

    push!(Master_response,cheMASTER.chemical_system_class(num_interaction,num_species,rxn_par,r_i,s_i))
    cheMASTER.initialize_dynamics(Master_response[i],initial_values,0,tf,0.1*dt)
    cheMASTER.runDynamics(Master_response[i],max_num=[10,10,20],method = "Euler",measureResponse="finite-cross",measureResponse_par=[1.1,1])
    
    println(i)

end
rxn_par[1] = k1

In [ ]:
# open("../data/gene/Master_response.jls", "w") do f
#     serialize(f, Master_response)
# end

In [ ]:
# open("../data/gene/master.jls", "r") do f
#     global Master_response = deserialize(f)
# end

In [ ]:
sat = 6

time_grid_all = EMRE[sat].timeGrid
time_grid_master = Master_response[sat].timeGrid
time_grid_gSBR = SOL[sat].t
plotting_tau =  [0.75, 1.5, 2.25, 3.0]

tau        = find_closed_timegrid_indices(time_grid_all, plotting_tau)
tau_master = find_closed_timegrid_indices(time_grid_master, plotting_tau)
tau_SBR    = find_closed_timegrid_indices(time_grid_gSBR, plotting_tau)

fig = figure(figsize=(15, 3.5))
ax1 = fig.add_axes([0, 0., 0.1, 1])
ax2 = fig.add_axes([0.11, 0., 0.19, 1])
ax3 = fig.add_axes([0.31, 0., 0.29, 1])
ax4 = fig.add_axes([0.61, 0., 0.38, 1])

axis = [ax4, ax3, ax2, ax1]

for j in 1:length(plotting_tau)

    axis[j].plot(time_grid_gSBR[tau_SBR[j]:end] .- time_grid_gSBR[tau_SBR[j]], (VAR[sat].R[3, 3, tau_SBR[j]:end, tau_SBR[j]]), "-", alpha=0.75,linewidth=3)

    axis[j].plot(time_grid_all[1:end-tau[j]], EMRE[sat].resp[3, 3, tau[j]+1:end, tau[j]], linestyle="dashdot", alpha=0.75,linewidth=3)

    #axis[j].plot(Master_response[sat].timeGrid[1:end-tau_master[j]], Master_response[sat].Response[3, 3, 1:end-tau_master[j], tau_master[j]], "--", alpha=0.75, color="gray",linewidth=3)
    
    axis[j].plot(Master_response[sat].timeGrid[tau_master[j]+1:end] .- Master_response[sat].timeGrid[tau_master[j]+1] , Master_response[sat].Response[3, 3, tau_master[j]+1:end, tau_master[j]], "--", alpha=0.75, color="gray",linewidth=3)

    #axis[j].plot(time_grid_all[1:end-tau[j]], respALL_2[2, 2, 1:end-tau[j], tau[j]], "-", alpha=1., linewidth=3)
    #axis[j].plot(time_grid_all[1:end-tau[j]], respLNA_2[2, 2, 1:end-tau[j], tau[j]], linestyle="dashdot", alpha=1., linewidth=3)
    #axis[j].plot(time_grid_all[1:end-tau[j]], respME_2[2, 2, 1:end-tau[j], tau[j]], "--", alpha=1., linewidth=3, color="gray")
    
    axis[j].set_title(L"\tau^\prime = " * string(plotting_tau[j]), fontsize=22)
    axis[j].spines["right"].set_visible(false)
    axis[j].spines["top"].set_visible(false)
    axis[j].set_xlabel(L"\tau-\tau\prime", fontsize=22)
    axis[j].set_ylim((-0.1, 1.))
end

ax1.set_ylabel(L"R_{PP}(\tau,\tau^\prime)", fontsize=22)
ax2.set(yticklabels=[])
ax3.set(yticklabels=[])
ax4.set(yticklabels=[])

ax4.legend(["gSBR", "LNA", "Master"], bbox_to_anchor=(0.45, 0.95), loc=0, frameon=false, fontsize=18)

plt.savefig("../plots/gene/gene_response_P.png", bbox_inches="tight")
plt.close()

gcf()


In [ ]:
close()

### Use normal moment closure using the MomentClosure package:

In [ ]:
using MomentClosure
# using DifferentialEquations
using OrdinaryDiffEq

raw_eqs = generate_raw_moment_eqs(gene_system, 2, combinatoric_ratelaws=false)

# Create normal closure system for gene regulation
closed_eqs = moment_closure(raw_eqs, "normal")

# Set up initial conditions for means and covariances
u0_map = [
    species(gene_system)[1] => 0.01,  # G
    species(gene_system)[2] => 1.0,   # G*
    species(gene_system)[3] => 0.1    # P
]

# Set up parameter mapping
p_map = [
    gene_system.k_t => 10.0,
    gene_system.k_b => 1.0,
    gene_system.k_u => 0.1,
    gene_system.k_2P => 2.0
]

# Solve for different k_u values
SOL_normal_moment_closure = []
for i in alpha_range
    # Update k_u parameter
    p_map_temp = copy(p_map)
    p_map_temp[3] = (gene_system.k_u => i)  # k_u is the 3rd parameter
    
    # Set up and solve ODE problem
    prob = ODEProblem(closed_eqs, u0_map, (tstart, tmax), p_map_temp)
    sol = solve(prob, Tsit5(), abstol=atol, reltol=rtol)
    
    push!(SOL_normal_moment_closure, sol)
end

In [ ]:
# open("../data/gene/sol_moment_closure.jls", "w") do f
#     serialize(f, SOL_normal_moment_closure)
# end

In [ ]:
open("../data/gene/sol_moment_closure.jls", "r") do f
    SOL_normal_moment_closure = deserialize(f)
end

#### Note that Moment clousre is unstable for this system for all k_u values.

## Plot comparison with other methods:

In [ ]:
plot_idx = 1
color = plt.rcParams["axes.prop_cycle"].by_key()["color"]
figure(figsize=(15,5))
for j in range(1,length(species(gene_system)))
    subplot(1,3,j)
    plot(SOL[plot_idx].t,VAR[plot_idx].μ[j,:],label="gSBR cross response",color=color[1],alpha=0.75,linestyle="--")
    plot(SOL_single_gSBR[plot_idx].t,VAR_single_gSBR[plot_idx].μ[j,:],label="gSBR single response",color=color[2],alpha=0.75,linestyle="--")
    plot(SOL_singleSBR[plot_idx].t,VAR_singleSBR[plot_idx].μ[j,:],label="SBR single response",color=color[3],alpha=0.75,linestyle="--")
    plot(SOL_crossMCA[plot_idx].t,VAR_crossMCA[plot_idx].μ[j,:],label="MCA cross response",color=color[4],alpha=0.75,linestyle="--")
    plot(SOL_singleMCA[plot_idx].t,VAR_singleMCA[plot_idx].μ[j,:],label="MCA single response",color=color[5],alpha=0.75,linestyle="--")
    # plot(SOL_mak[plot_idx].t, VAR_mak[plot_idx].μ[j,:],label="MAK",color=color[6],alpha=0.5,linestyle="--")
    # plot(EMRE[plot_idx].timeGrid, EMRE[plot_idx].y[j,:],label="EMRE",color=color[7],alpha=0.5,linestyle="--")
    plot(Master[plot_idx].timeGrid, Master[plot_idx].y[j,:],label="Master Eq.",color=color[8],alpha=0.75,linestyle="-")
    if j == 1
        title("G")
    elseif j == 2
        title("G*")
    else
        title("P")
    end
    yscale("log")
    xlabel(L"$\tau$")
    ylabel(L"$\langle n(\tau)\rangle$")
end
legend()

gcf()

In [ ]:
savefig("../plots/gene/gene_means_comparisons_all_methods_noMAK.png", bbox_inches="tight")
close()

In [ ]:
plot_idx = 1
color = plt.rcParams["axes.prop_cycle"].by_key()["color"]
figure(figsize=(15,5))
for j in range(1,length(species(gene_system)))
    subplot(1,3,j)
    plot(SOL[plot_idx].t,VAR[plot_idx].μ[j,:],label="gSBR cross response",color=color[1],alpha=0.75,linestyle="--")
    plot(SOL_single_gSBR[plot_idx].t,VAR_single_gSBR[plot_idx].μ[j,:],label="gSBR single response",color=color[2],alpha=0.75,linestyle="--")
    plot(SOL_singleSBR[plot_idx].t,VAR_singleSBR[plot_idx].μ[j,:],label="SBR single response",color=color[3],alpha=0.75,linestyle="--")
    plot(SOL_crossMCA[plot_idx].t,VAR_crossMCA[plot_idx].μ[j,:],label="MCA cross response",color=color[4],alpha=0.75,linestyle="--")
    plot(SOL_singleMCA[plot_idx].t,VAR_singleMCA[plot_idx].μ[j,:],label="MCA single response",color=color[5],alpha=0.75,linestyle="--")
    plot(SOL_mak[plot_idx].t, VAR_mak[plot_idx].μ[j,:],label="MAK",color=color[6],alpha=0.75,linestyle="--")
    plot(EMRE[plot_idx].timeGrid, EMRE[plot_idx].y[j,:],label="EMRE",color=color[7],alpha=0.75,linestyle="--")
    plot(Master[plot_idx].timeGrid, Master[plot_idx].y[j,:],label="Master Eq.",color=color[8],alpha=0.75,linestyle="-")
    if j == 1
        title("G")
    elseif j == 2
        title("G*")
    else
        title("P")
    end
    yscale("log")
    xlabel(L"$\tau$")
    ylabel(L"$\langle n(\tau)\rangle$")
end
legend()

gcf()

In [ ]:
savefig("../plots/gene/gene_means_comparisons_all_methods.png", bbox_inches="tight")
close()

In [ ]:
plot_idx = 1
color = plt.rcParams["axes.prop_cycle"].by_key()["color"]
figure(figsize=(15,5))
for j in range(1,length(species(gene_system)))
    subplot(1,3,j)
    plot(SOL[plot_idx].t, diag(VAR[plot_idx].N[j,j,:,:]),label="gSBR cross response",color=color[1],alpha=0.75,linestyle="--")
    plot(SOL_single_gSBR[plot_idx].t,diag(VAR_single_gSBR[plot_idx].N[j,:,:]),label="gSBR single response",color=color[2],alpha=0.75,linestyle="--")
    plot(SOL_singleSBR[plot_idx].t,diag(VAR_singleSBR[plot_idx].N[j,:,:]),label="SBR single response",color=color[3],alpha=0.75,linestyle="--")
    plot(SOL_crossMCA[plot_idx].t,diag(VAR_crossMCA[plot_idx].N[j,j,:,:]),label="MCA cross response",color=color[4],alpha=0.75,linestyle="--")
    plot(SOL_singleMCA[plot_idx].t,diag(VAR_singleMCA[plot_idx].N[j,:,:]),label="MCA single response",color=color[5],alpha=0.75,linestyle="--")
    # plot(SOL_mak[plot_idx].t, diag(VAR_mak[plot_idx].N[j,:,:]),label="MAK",color=color[6],alpha=0.5,linestyle="--")
    # plot(EMRE[plot_idx].timeGrid, EMRE[plot_idx].lna_var[j,j,:],label="EMRE",color=color[7],alpha=0.5,linestyle="--")
    plot(Master[plot_idx].timeGrid, Master[plot_idx].variance[j,:],label="Master Eq.",color=color[8],alpha=0.75,linestyle="-")
    if j == 1
        title("G")
    elseif j == 2
        title("G*")
    else
        title("P")
    end
    yscale("log")
    xlabel(L"$\tau$")
    ylabel(L"$\langle \delta n(\tau) \delta n(\tau) \rangle$")
end
legend()

gcf()

In [ ]:
savefig("../plots/gene/gene_variances_comparisons_all_methods_noMAK.png", bbox_inches="tight")
close()

In [ ]:
plot_idx = 1
color = plt.rcParams["axes.prop_cycle"].by_key()["color"]
figure(figsize=(15,5))
for j in range(1,length(species(gene_system)))
    subplot(1,3,j)
    plot(SOL[plot_idx].t, diag(VAR[plot_idx].N[j,j,:,:]),label="gSBR cross response",color=color[1],alpha=0.75,linestyle="--")
    plot(SOL_single_gSBR[plot_idx].t,diag(VAR_single_gSBR[plot_idx].N[j,:,:]),label="gSBR single response",color=color[2],alpha=0.75,linestyle="--")
    plot(SOL_singleSBR[plot_idx].t,diag(VAR_singleSBR[plot_idx].N[j,:,:]),label="SBR single response",color=color[3],alpha=0.75,linestyle="--")
    plot(SOL_crossMCA[plot_idx].t,diag(VAR_crossMCA[plot_idx].N[j,j,:,:]),label="MCA cross response",color=color[4],alpha=0.75,linestyle="--")
    plot(SOL_singleMCA[plot_idx].t,diag(VAR_singleMCA[plot_idx].N[j,:,:]),label="MCA single response",color=color[5],alpha=0.75,linestyle="--")
    plot(SOL_mak[plot_idx].t, diag(VAR_mak[plot_idx].N[j,:,:]),label="MAK",color=color[6],alpha=0.75,linestyle="--")
    plot(EMRE[plot_idx].timeGrid, EMRE[plot_idx].lna_var[j,j,:],label="EMRE",color=color[7],alpha=0.75,linestyle="--")
    plot(Master[plot_idx].timeGrid, Master[plot_idx].variance[j,:],label="Master Eq.",color=color[8],alpha=0.75,linestyle="-")
    if j == 1
        title("G")
    elseif j == 2
        title("G*")
    else
        title("P")
    end
    yscale("log")
    xlabel(L"$\tau$")
    ylabel(L"$\langle \delta n(\tau) \delta n(\tau) \rangle$")
end
legend()

gcf()

In [ ]:
savefig("../plots/gene/gene_variances_comparisons_all_methods.png", bbox_inches="tight")
close()